In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import ImageFolder

import os
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

from kagglehub import dataset_download

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
path = dataset_download("puneet6060/intel-image-classification")
print("Path to dataset files:", path)

train_path = os.path.join(path, "seg_train", "seg_train")
test_path = os.path.join(path, "seg_test", "seg_test")

print("Classes in training folder:", os.listdir(train_path))

train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

batch_size = 64
num_workers = 6

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print("✅ Dataset loaded successfully!")
print("Classes:", train_dataset.classes)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

The `transform` function is related to how to address image data. The following numbered bullets are about explanations;

1. **`transforms.Resize((150, 150))`**: Resizes the input image to a fixed size of 150x150 pixels. This ensures all images have the same dimensions (although the original dataset gave its image size is 150x150), allowing the neural network model to train uniformly among various images.

2. **`transforms.ToTensor()`**: Converts the image from a PIL image or NumPy array into a PyTorch tensor. It also scales the pixel values from the range [0, 255] to [0, 1].

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out

In [ ]:
class CustomSceneCNN(nn.Module):
    def __init__(self):
        super(CustomSceneCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Original 8 layers
        self.layer1 = ResidualBlock(16, 16)
        self.layer2 = ResidualBlock(16, 32, stride=2)
        self.layer3 = ResidualBlock(32, 32)
        self.layer4 = ResidualBlock(32, 64, stride=2)
        self.layer5 = ResidualBlock(64, 64)
        self.layer6 = ResidualBlock(64, 128, stride=2)
        self.layer7 = ResidualBlock(128, 128)
        self.layer8 = ResidualBlock(128, 256, stride=2)
        self.layer9 = ResidualBlock(256, 256)
        self.layer10 = ResidualBlock(256, 512, stride=2)
        self.layer11 = ResidualBlock(512, 512)
        self.layer12 = ResidualBlock(512, 1024, stride=2)
        self.layer13 = ResidualBlock(1024, 1024)
        self.layer14 = ResidualBlock(1024, 2048, stride=2)
        self.layer15 = ResidualBlock(2048, 2048)
        self.layer16 = ResidualBlock(2048, 4096, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))


        self.fc1 = nn.Linear(4096, 1024)
        self.fc_bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 64)
        self.fc_bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 6)  # Output layer for 6 classes

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))

        # Original 8 layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        
        # Additional 8 layers
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layer13(x)
        x = self.layer14(x)
        x = self.layer15(x)
        x = self.layer16(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        # Updated FC layer forward pass
        x = self.relu(self.fc_bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.fc_bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.relu(self.fc_bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)

        return x

In [ ]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        outputs = model(data)
        loss = criterion(outputs, target)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    return train_loss, train_accuracy

In [ ]:
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            test_loss += criterion(outputs, target).item()
            _, predicted = outputs.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / total

    return test_loss, test_accuracy, all_preds, all_targets

In [ ]:
model_path = './intel_scene_cnn_model.pth'
history_path = './intel_scene_cnn_history.pth'
model = CustomSceneCNN().to(device)

if os.path.exists(model_path):
    train_time_compute = 0
    print(f"Loading pre-trained model from {model_path}")
    model.load_state_dict(torch.load(model_path, weights_only=True))

    if os.path.exists(history_path):
        history = torch.load(history_path)
        train_losses = history['train_losses']
        train_accuracies = history['train_accuracies']
        test_losses = history['test_losses']
        test_accuracies = history['test_accuracies']
        print(f"Loaded training history for {len(train_losses)} epochs")
    else:
        print("No training history found. Evaluating current model performance...")
        test_loss, test_accuracy, _, _ = test(model, test_loader, criterion, device)
        print(f'Loaded Model Test Accuracy: {test_accuracy:.2f}%')
        train_losses = [0]
        test_losses = [test_loss]
        train_accuracies = [0]
        test_accuracies = [test_accuracy]
else:
    print("No previously trained model found. Training from scratch...")
    train_time_compute = 0
    start_time = time.time()
    max_epochs = 20
    patience = 3
    best_accuracy = 0
    no_improve_count = 0

    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(max_epochs):
        train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
        test_loss, test_accuracy, _, _ = test(model, test_loader, criterion, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        print(f'Epoch {epoch+1}/{max_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
              f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(model.state_dict(), model_path)
            print(f"Model improved, saved to {model_path}")
            no_improve_count = 0
        else:
            no_improve_count += 1
            print(f"No improvement for {no_improve_count} epochs")

        history = {
            'train_losses': train_losses,
            'train_accuracies': train_accuracies,
            'test_losses': test_losses,
            'test_accuracies': test_accuracies
        }
        torch.save(history, history_path)

        if no_improve_count >= patience:
            print(f"Early stopping after {epoch+1} epochs")
            break

    end_time = time.time()
    training_time = end_time - start_time
    print(f'Training completed in {training_time:.2f} seconds')

    model.load_state_dict(torch.load(model_path, weights_only=True))
    print(f"Loaded best model with accuracy: {best_accuracy:.2f}%")
    print(f"Training history saved to {history_path}")
